## Simulate running on multiple GPUs

In [1]:
from dask.distributed import Client

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.chdir('/home/ubuntu/adadamp-experiments/')

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import csv
import importlib
import pandas as pd
from time import time
from copy import copy

In [4]:
# import simulator
os.chdir('/home/ubuntu/adadamp-experiments/')
classifier = importlib.import_module("exp-dask.classifier")
from classifier import DaskClassifierSimulator

In [5]:
# training client
from dask.distributed import Client

def _prep():
    from distributed.protocol import torch

client = Client(processes=False)
client.run(_prep)
client

Client Scheduler: inproc://172.31.40.124/4380/1 Dashboard: http://172.31.40.124:8787/status,Cluster Workers: 1 Cores: 4 Memory: 16.48 GB


In [6]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=True, download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=False, download=True, transform=transform_test)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [7]:
from model import Wide_ResNet

client.upload_file("./exp-dask/model.py")

In [8]:
def write_stats(hist, directory, filename):
    toCSV = hist
    with open('./exp-dask/{}/{}'.format(directory, filename), 'w', encoding='utf8', newline='') as output_file:
        fc = csv.DictWriter(output_file, fieldnames=toCSV[0].keys())
        fc.writeheader()
        fc.writerows(toCSV)

In [9]:
def train(model, train_set, test_set, stats_df, n_epochs=200, log_interval=1, stats_dir=''):
    """
    Train based on expiriment params
    
    Parameters:
    epoch_sched: update lr and bs at epochs in this list
    lr_sched: update lr to value at matching epoch. Should be same length as epoch_sched
    bs_sched: update bs to value at matching epoch. Should be same length as epoch_sched
    """
    
    stats = list(stats_df.T.to_dict().values())
    print("[SETUP] Loaded {} epochs of stats".format(len(stats)))
    
    # preprocess data
    start = time()
    train_set = model.preprocess(train_set)
    test_set = model.preprocess(test_set)
    print("[SETUP] Pre-Processed in {} seconds".format(time() - start))
    
    history = []
    for epoch in range(min(n_epochs, len(stats))):
        
        start = time()
        
        # set stats
        stat = stats.pop(0)
        model.set_sim(stat)
        
        # run
        print("[Epoch {}]".format(epoch), end="")
        model.partial_fit(train_set)
        score = model.score(test_set)
        datum = {"epoch": epoch, "score": score, **model.get_params(), **model.meta_}
        print(" Score: {} in {} seconds".format(score, time() - start))
        history.append(datum)
        
        if epoch % log_interval == 0:
            write_stats(history, stats_dir, 'results-ep{}.csv'.format(epoch))

    return history

In [10]:
def run_sim(stats_df_loc, out_folder, timings):
    
    print("Running simulation....\n\n")
    start = time()
    
    stats_df = pd.read_csv(stats_df_loc)
    
    if 'lr_' in stats_df.columns:
        stats_df = stats_df.rename(columns={"lr_": "partial_fit__lr", "damping_": "partial_fit__batch_size"})
    
    init_lr = stats_df['partial_fit__lr'][0]
    momentum = stats_df['optimizer__momentum'][0]
    device = "cpu" if not torch.cuda.is_available() else "cuda:0"
    
    args = dict(
        module=Wide_ResNet,
        module__depth=16,
        module__widen_factor=4,
        module__dropout_rate=0.3,
        module__num_classes=len(classes),
        loss=torch.nn.CrossEntropyLoss,
        optimizer=torch.optim.SGD,
        optimizer__lr=init_lr,
        optimizer__momentum=momentum,
        optimizer__nesterov=True,
        optimizer__weight_decay=0.5e-3,
        batch_size=128,
        max_epochs=200,
        device=device,
        grads_per_worker=128,
        client=client
    )
    model = DaskClassifierSimulator(**args)
    model.set_times(timings['score'], timings['deepcopy'], timings['grad128'])
    
    # train
    hist = train(model, 
             train_set, 
             test_set, 
             stats_df,
             n_epochs=200, 
             log_interval=20,
             stats_dir=out_folder
            )
    write_stats(hist, out_folder, 'results-final.csv')
    
    print('Finished simulation in {} seconds'.format(time() - start))
    

In [ ]:
# test exp
timings = { 'score': 0.0, 'deepcopy': 0.0, 'grad128': 0.0 }
stats_path = './exp-dask/stats/decreasing-lr/exp--final.csv'
out_path = 'sim-results/test'
run_sim(stats_path, out_path, timings)

In [12]:
## TIMINGS FOR NORMAL EXPERIMENTS
SCORE_TIME = 0.0 # 5.39407711148262
DEEPCOPY_TIME = 0.05855  # seconds
GRAD_TIME_128 = 0.07832  # seconds
timings = { 'score': SCORE_TIME, 'deepcopy': DEEPCOPY_TIME, 'grad128': GRAD_TIME_128 }

In [14]:
# exp 1 - dec-lr-1-machine 
stats_path = './exp-dask/stats/decreasing-lr/exp--final.csv'
out_path = 'sim-results/dec-lr-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 17.791621685028076 seconds
[Epoch 0] Score: 0.5990999937057495 in 60.93469595909119 seconds
[Epoch 1] Score: 0.6762999892234802 in 60.52432322502136 seconds
[Epoch 2] Score: 0.7156999707221985 in 60.424665451049805 seconds
[Epoch 3] Score: 0.7777999639511108 in 60.22819423675537 seconds
[Epoch 4] Score: 0.7678999900817871 in 60.64512586593628 seconds
[Epoch 5] Score: 0.7994999885559082 in 60.43351221084595 seconds
[Epoch 6] Score: 0.8030999898910522 in 60.24355626106262 seconds
[Epoch 7] Score: 0.8100000023841858 in 60.417123794555664 seconds
[Epoch 8] Score: 0.8186999559402466 in 60.66122794151306 seconds
[Epoch 9] Score: 0.8077999949455261 in 60.36531925201416 seconds
[Epoch 10] Score: 0.8211999535560608 in 60.62680649757385 seconds
[Epoch 11] Score: 0.8321999907493591 in 60.30172610282898 seconds
[Epoch 12] Score: 0.8306999802589417 in 60.565566539764404 seconds
[Epoch 13] Score: 0.8325999975204468 

In [15]:
# exp 2 - dec-lr-1-machine
stats_path = './exp-dask/stats/increasing-bs/exp-final.csv'
out_path = 'sim-results/inc-bs-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 17.47501015663147 seconds
[Epoch 0] Score: 0.5943999886512756 in 60.34588360786438 seconds
[Epoch 1] Score: 0.6757999658584595 in 59.81830382347107 seconds
[Epoch 2] Score: 0.7128999829292297 in 60.05758547782898 seconds
[Epoch 3] Score: 0.7423999905586243 in 60.13798546791077 seconds
[Epoch 4] Score: 0.7707999944686891 in 59.78105878829956 seconds
[Epoch 5] Score: 0.7875999808311462 in 60.20547914505005 seconds
[Epoch 6] Score: 0.7892999649047852 in 60.13836622238159 seconds
[Epoch 7] Score: 0.8071999549865723 in 60.33895206451416 seconds
[Epoch 8] Score: 0.8120999932289124 in 59.6184241771698 seconds
[Epoch 9] Score: 0.8226000070571899 in 59.78698968887329 seconds
[Epoch 10] Score: 0.8353999853134155 in 59.83977770805359 seconds
[Epoch 11] Score: 0.8224999904632568 in 59.69212603569031 seconds
[Epoch 12] Score: 0.8246999979019165 in 59.85038948059082 seconds
[Epoch 13] Score: 0.8355000019073486 in 59

In [16]:
# exp 3 - hybrid 1 1 machine
stats_path = './exp-dask/stats/hybrid/exp-final.csv'
out_path = 'sim-results/hybrid1-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.63489079475403 seconds
[Epoch 0] Score: 0.6014999747276306 in 60.31620526313782 seconds
[Epoch 1] Score: 0.6804999709129333 in 59.910706520080566 seconds
[Epoch 2] Score: 0.7328000068664551 in 59.728004693984985 seconds
[Epoch 3] Score: 0.7621999979019165 in 59.67918539047241 seconds
[Epoch 4] Score: 0.7834999561309814 in 60.114511013031006 seconds
[Epoch 5] Score: 0.8009999990463257 in 60.12607407569885 seconds
[Epoch 6] Score: 0.8064000010490417 in 60.05822443962097 seconds
[Epoch 7] Score: 0.8100000023841858 in 59.77878212928772 seconds
[Epoch 8] Score: 0.8197999596595764 in 59.69206953048706 seconds
[Epoch 9] Score: 0.812999963760376 in 59.98140001296997 seconds
[Epoch 10] Score: 0.8154999613761902 in 60.13196301460266 seconds
[Epoch 11] Score: 0.8373000025749207 in 60.33780097961426 seconds
[Epoch 12] Score: 0.8402000069618225 in 60.300156116485596 seconds
[Epoch 13] Score: 0.8319000005722046 i

In [17]:
# exp 4 - hybrid 2 1 machine
stats_path = './exp-dask/stats/hybrid-2/exp-final.csv'
out_path = 'sim-results/hybrid2-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 19.280600786209106 seconds
[Epoch 0] Score: 0.6290000081062317 in 60.20195508003235 seconds
[Epoch 1] Score: 0.694599986076355 in 59.73836946487427 seconds
[Epoch 2] Score: 0.7508999705314636 in 59.90464544296265 seconds
[Epoch 3] Score: 0.7793999910354614 in 59.96568727493286 seconds
[Epoch 4] Score: 0.8016999959945679 in 59.94650745391846 seconds
[Epoch 5] Score: 0.8024999499320984 in 60.222100496292114 seconds
[Epoch 6] Score: 0.8096999526023865 in 60.144551277160645 seconds
[Epoch 7] Score: 0.8320999741554259 in 59.60941958427429 seconds
[Epoch 8] Score: 0.8307999968528748 in 59.995556354522705 seconds
[Epoch 9] Score: 0.8369999527931213 in 59.66912841796875 seconds
[Epoch 10] Score: 0.8292999863624573 in 59.77865028381348 seconds
[Epoch 11] Score: 0.8430999517440796 in 59.69939947128296 seconds
[Epoch 12] Score: 0.8443999886512756 in 59.76674795150757 seconds
[Epoch 13] Score: 0.8333999514579773 i

In [18]:
# exp 5 - large bs 1 1 machine
stats_path = './exp-dask/stats/large-bs-0/exp-final.csv'
out_path = 'sim-results/large-bs1-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 18.32858681678772 seconds
[Epoch 0] Score: 0.5884999632835388 in 60.19100999832153 seconds
[Epoch 1] Score: 0.6516000032424927 in 60.15177035331726 seconds
[Epoch 2] Score: 0.7285000085830688 in 60.04320955276489 seconds
[Epoch 3] Score: 0.7662000060081482 in 59.7692768573761 seconds
[Epoch 4] Score: 0.7896999716758728 in 59.880232095718384 seconds
[Epoch 5] Score: 0.7972999811172485 in 60.19983696937561 seconds
[Epoch 6] Score: 0.8024999499320984 in 60.13681721687317 seconds
[Epoch 7] Score: 0.8089999556541443 in 60.09073352813721 seconds
[Epoch 8] Score: 0.8148999810218811 in 60.041139125823975 seconds
[Epoch 9] Score: 0.8120999932289124 in 60.04808163642883 seconds
[Epoch 10] Score: 0.8294000029563904 in 60.391865730285645 seconds
[Epoch 11] Score: 0.8185999989509583 in 59.79838538169861 seconds
[Epoch 12] Score: 0.8331999778747559 in 59.94559931755066 seconds
[Epoch 13] Score: 0.8416999578475952 in

In [ ]:
# exp 6 - large bs 2 1 machine
stats_path = './exp-dask/stats/large-bs-1/exp-final.csv'
out_path = 'sim-results/large-bs2-1-machine'
run_sim(stats_path, out_path, timings)

Running simulation....


[SETUP] Loaded 200 epochs of stats
[SETUP] Pre-Processed in 19.15642476081848 seconds
[Epoch 0] Score: 0.4782999753952026 in 59.93417501449585 seconds
[Epoch 1] Score: 0.5977999567985535 in 59.8633987903595 seconds
[Epoch 2] Score: 0.665399968624115 in 59.497079849243164 seconds
[Epoch 3] Score: 0.6798999905586243 in 60.07591652870178 seconds
[Epoch 4] Score: 0.6807999610900879 in 60.03966975212097 seconds
[Epoch 5] Score: 0.7182999849319458 in 60.55995059013367 seconds
[Epoch 6] Score: 0.7193999886512756 in 59.96700644493103 seconds
[Epoch 7] Score: 0.7482999563217163 in 59.8819580078125 seconds
[Epoch 8] Score: 0.6961999535560608 in 59.726155281066895 seconds
[Epoch 9] Score: 0.7603999972343445 in 60.03394317626953 seconds
[Epoch 10] Score: 0.7512999773025513 in 59.92246103286743 seconds
[Epoch 11] Score: 0.7337999939918518 in 60.03620195388794 seconds
[Epoch 12] Score: 0.7601000070571899 in 59.76226782798767 seconds
[Epoch 13] Score: 0.7540000081062317 in 59

In [ ]:
# exp 7 - dec-lr-P-machine 

In [ ]:
# exp 8 - dec-lr-P-machine

In [ ]:
# exp 9 - hybrid 1 P machine

In [ ]:
# exp 10 - hybrid 2 P machine

In [ ]:
# exp 11 - large bs 1 P machine

In [ ]:
# exp 12 - large bs 2 P machine